In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = None,                 # Elongation factor
    delta       = None,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04                  # position of the LCFS (x_inner)
)

simulation.set_data_param(
    expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/',
    g0simdir   = home_dir+'/g0_sim/',
    simname    = '48x32x16x16x6_nunifvg',
    simdir     = 'gk_tcv_posD_iwl_3x2v_AH/',
    fileprefix = 'gk_tcv_posD_iwl_3x2v_D02',
    wkdir      = 'wk/'
)

# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)
# Check available frames
tfs = find_available_frames(simulation,'field')
print(tfs)

#---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','LCFS')
simulation.normalize('y','Larmor radius')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','beta')

In [ ]:
#---------------- Profile plots
#-- Plane location
cdirection='x'; ccoords=[0.0,0]
# cdirection='y'; ccoords=[0.0,0.0]
# cdirection='z'; ccoords=[0.045,0.0]
#----------------
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldname  = 'pperp', # field to plot (full_plot is prior)
                       twindow    = tfs,     # time window to plot
                       space_time = True,    # plot as space time diagram
                       time_avg   = False,   # time average the profile (prior on space_time)
                       full_plot  = False,   # 
                       spec       = 'e')

In [ ]:
#---------- 2D cut plot
#-- choose min and max time frames and cut location
cplane='yz'; ccoord=0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = 110, 
            fieldname = '',
            cmap = 'bwr',
            spec = 'e')

In [ ]:
#---------- 2D cut movie
#-- choose min and max time frames and cut location
cplane='xy'; ccoord=0
make_2D_movie(simulation, cplane, ccoord, 
            tfs = tfs[-50:], 
            fieldname = 'Tpari',
            spec = 'e',
            cmap = 'bwr'
            # clim=[0, 200],
            )

In [ ]:
if False:
    #---------------- 2D vpar x plots
    # Simulation frame
    tf = 0
    # Data to plot
    spec = elc
    #spec = ion
    yf = 0.0
    zf = 0.0
    mf = 0.0
    #----------------

    fname = simulation.get_filename('f'+spec.nshort,tf)
    data = pg.data.GData(fname)
    dg = pg.data.GInterpModal(data,1,'ms')
    dg.interpolate(0,overwrite=True)
    pg.data.select(data, z1=yf, z2=zf, z4=mf, overwrite=True)
    pg.output.plot(data, yscale=1./spec.vt,
                title=r"y=%2.2f, z=%2.2f, $\mu$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                contour=False,diverging=False, xshift=-x_LCFS,
                xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$v_\parallel/v_{th%s}$'%spec.nshort,
                clabel=r'$f_{%s}$'%spec.nshort);

In [ ]:
if False:
    #---------------- 2D mu x plots
    # Simulation frame
    tf = 0
    # Data to plot
    spec = elc
    #spec = ion
    yf = 0.0
    zf = 0.0
    vf = 0.0
    #----------------

    fname = simulation.get_filename('f'+spec.nshort,tf)
    data = pg.data.GData(fname)
    dg = pg.data.GInterpModal(data,1,'ms')
    dg.interpolate(0,overwrite=True)
    pg.data.select(data, z1=yf, z2=zf, z3=vf, overwrite=True)
    pg.output.plot(data, yscale=1./spec.mu0,
                title=r"y=%2.2f, z=%2.2f, $\mu$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                contour=False,diverging=False, xshift=-x_LCFS,
                xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$\mu B/T_{%s}$'%spec.nshort,
                clabel=r'$f_{%s}$'%spec.nshort);